In [1]:
import optuna

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from stesml.model_tools import build_train_test_model

/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [3]:
def objective(trial):   
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 10, 10000, log=True)
        epochs = trial.suggest_int("epochs", 1, 20)
        result = build_train_test_model(data_dir, model_type, target, metric, scale, n_layers, n_hidden_units, batch_size, epochs, n_shuffles=n_shuffles)
    elif model_type == 'XGBoost':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 50, 10000, log=True)
        result = build_train_test_model(data_dir, model_type, target, metric, scale, n_estimators=n_estimators, n_shuffles=n_shuffles)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 100, log=True)
        result = build_train_test_model(data_dir, model_type, target, metric, scale, n_estimators=n_estimators, n_shuffles=n_shuffles)
    
    return result

In [4]:
data_dir = "../data/Sulfur_Models/"
model_type = 'NN' # Options: NN, XGBoost, RandomForest
target = 'Tavg' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_shuffles = 5

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'

In [5]:
study = optuna.create_study(direction=direction)

[I 2022-06-29 17:39:57,104] A new study created in memory with name: no-name-c7207410-815f-44ea-8f77-160146b7ef4c


In [ ]:
study.optimize(objective, n_trials=50)

2022-06-29 17:40:13.864735: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-29 17:40:13.982424: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


Epoch 1/14
46015/46015 [==============================] - 38s 829us/step - loss: 0.0057 - val_loss: 0.0012
Epoch 2/14
46015/46015 [==============================] - 40s 867us/step - loss: 1.1819e-04 - val_loss: 0.0013
Epoch 3/14
46015/46015 [==============================] - 31s 665us/step - loss: 1.0074e-04 - val_loss: 0.0014
Result: 1.9130
Average Result: 1.9130
Epoch 1/14
46030/46030 [==============================] - 42s 905us/step - loss: 0.0106 - val_loss: 0.0015
Epoch 2/14
28582/46030 [=================>............] - ETA: 12s - loss: 1.5076e-04

In [17]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621